In [2]:
!pip install pandas
!pip install html5lib
!pip install lxml
!pip install BeautifulSoup4

In [3]:
from pandas.io.html import read_html

Getting all tables from the wikipedia page (in this case there's only one)

In [4]:
page = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

wikitables = read_html(page, attrs=("class","wikitable"))
print ("Extracted {num} wikitables".format(num=len(wikitables)))

Extracted 1 wikitables


In [5]:
wikitables[0].head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [6]:
wikitables[0].shape

(287, 3)

In [7]:
df_Toronto = wikitables[0]
df_Toronto.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Dropping any rows with "Not assigned" in the Borough column

In [8]:
# Get names of indexes for which column Neighbourhood has value "Not assigned"
indexN = df_Toronto[ df_Toronto['Borough'] == "Not assigned" ].index
 
# Delete these row indexes from dataFrame
df_Toronto.drop(indexN , inplace=True)

In [9]:
df_Toronto.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


Combining neighbourhoods that all have the same postcode into one row & re-indexing

In [10]:
df_T = df_Toronto.groupby(["Postcode", "Borough"])["Neighbourhood"].apply(", ".join).to_frame(name = 'Neighbourhood').reset_index()

In [11]:
df_T.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [12]:
df_T.shape

(103, 3)

If any neighbourhoods are "Not assigned" I replace them with NaN and then use ffill to fill them with the value from the previous column, in this case the Borough.

In [13]:
import numpy as np
df_T.replace(to_replace ="Not assigned", value = np.nan) 

df_T.ffill(axis=1)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


In [14]:
df_T.tail(20)

,Postcode,Borough,Neighbourhood
83,M6R,West Toronto,"Parkdale, Roncesvalles"
84,M6S,West Toronto,"Runnymede, Swansea"
85,M7A,Downtown Toronto,Queen's Park
86,M7R,Mississauga,Canada Post Gateway Processing Centre
87,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern
88,M8V,Etobicoke,"Humber Bay Shores, Mimico South, New Toronto"
89,M8W,Etobicoke,"Alderwood, Long Branch"
90,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
91,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So..."
92,M8Z,Etobicoke,"Kingsway Park South West, Mimico NW, The Queen..."


In [15]:
df_T.shape

(103, 3)

Getting the geographic coordinates

In [27]:
import pandas as pd
df_geo = pd.read_csv('http://cocl.us/Geospatial_data')

print ('Data read into a pandas dataframe!')

Data read into a pandas dataframe!


In [49]:
df_geo.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Joining latitude and longitude to the postcodes and neighbourhoods dataframe

In [41]:
df_geo.rename(columns={'Postal Code':'Postcode'}, inplace=True)
df_geo.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [50]:
import pandas as pd
df_T.merge(df_geo, on='Postcode')

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437
